In [1]:
import numpy as np

def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def sigmoid_deriv(x):
    return sigmoid(x) * (1-sigmoid(x))

In [2]:
# data I/O
data = open('input.txt', 'r').read()  # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

data has 1115394 characters, 65 unique.


In [3]:
# define model
# hyperparameters

h = 100 # size of hidden layer of neurons
seq_length = 25  # number of steps to unroll the RNN
learning_rate = 5e-2

# model parameters
Wx = np.random.randn( 4 * h, vocab_size) * 0.01
Wh = np.random.randn( 4 * h, h) * 0.01
Wy = np.random.randn(vocab_size, h) * 0.01
b = np.zeros((4 * h, 1))
by = np.zeros((vocab_size, 1))

In [4]:
def lossFun(inputs, targets, hprev, cprev):
    
    xs, hs, ys, ps, cs, i, f, o, g, tmp = {}, {}, {}, {}, {}, {}, {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    cs[-1] = np.copy(cprev)
    loss = 0
    # forward pass
    for t in range(len(inputs)):
    
        xs[t] = np.zeros((vocab_size, 1))  # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        tmp[t] = np.dot(Wx, xs[t]) + np.dot(Wh, hs[t - 1]) + b
        f[t] = sigmoid(tmp[t][0:h])
        i[t] = sigmoid(tmp[t][h:2*h])
        g[t] = np.tanh(tmp[t][2*h:3*h])
        o[t] = sigmoid(tmp[t][3*h::])
        cs[t] = f[t]*cs[t-1] + i[t]*g[t]
        hs[t] = o[t] * np.tanh(cs[t])
        ys[t] = np.dot(Wy, hs[t]) + by
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))  # probabilities for next chars
        loss += -np.log(ps[t][targets[t], 0])
        
    #backward pass
    dWh, dWx, dWy = np.zeros_like(Wh), np.zeros_like(Wx), np.zeros_like(Wy)
    db, dby = np.zeros_like(b), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    dcnext = np.zeros_like(cs[0])
    for t in reversed(range(len(inputs))):
        
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1
        dWy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Wy.T, dy) + dhnext
        
        dtmp = np.zeros((4*h, 1)) 
        dc = dcnext + o[t] *(np.cosh(cs[t])**-2) * dh
        do = np.tanh(cs[t]) * dh
        dg = i[t] * dc
        di = g[t] * dc
        df = cs[t-1]* dc 
        
        dtmp[0:h] = df * sigmoid_deriv(tmp[t][0:h])
        dtmp[h:2*h] = di * sigmoid_deriv(tmp[t][h:2*h])
        dtmp[2*h:3*h] = dg * np.cosh(tmp[t][2*h:3*h])**-2
        dtmp[3*h::] = do * sigmoid_deriv(tmp[t][3*h::])
        
        db = np.sum(dtmp, axis=0)
        dWx = np.dot(dtmp,xs[t].T)#cx
        dWh = np.dot(dtmp,hs[t-1].T)#only t
        
        dhnext = np.dot(Wh.T, dtmp)
        dcnext = dc * f[t]
        
    return loss, dWx, dWh, dWy, db, dby, hs[len(inputs) - 1], cs[len(inputs) - 1]
        

In [5]:
def sample(h, seed_ix, n, c):
    """
    sample a sequence of integers from the model
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    hn = h.shape[0]
    for t in range(n):
        
        tmp = np.dot(Wx, x) + np.dot(Wh, h) + b
        f = sigmoid(tmp[0:hn])
        i = sigmoid(tmp[hn:2*hn])
        g = np.tanh(tmp[2*hn:3*hn])
        o = sigmoid(tmp[3*hn::])
        
        c = f*c + i*g
        h = o * np.tanh(c)
        y = np.dot(Wy, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

In [6]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(Wy)
mbh, mby = np.zeros_like(b), np.zeros_like(by)  # memory variables for Adagrad
smooth_loss = -np.log(1.0 / vocab_size) * seq_length  # loss at iteration 0
while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p + seq_length + 1 >= len(data) or n == 0:
        hprev = np.zeros((h, 1))  # reset RNN memory
        cprev = np.zeros((h, 1))
        p = 0  # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p + seq_length]]
    targets = [char_to_ix[ch] for ch in data[p + 1:p + seq_length + 1]]

    # sample from the model now and then
    if n % 1000 == 0:
        sample_ix = sample(hprev, inputs[0], 200,cprev)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt,))

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev, cprev = lossFun(inputs, targets, hprev, cprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 1000 == 0: print
    'iter %d, loss: %f' % (n, smooth_loss)  # print progress

    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wx, Wh, Wy, b, by],
                                  [dWxh, dWhh, dWhy, dbh, dby],
                                  [mWxh, mWhh, mWhy, mbh, mby]):
        
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8)  # adagrad update

    p += seq_length  # move data pointer
    n += 1  # iteration counter

----
 GfYCUnkxK PcBu.C
Nop$U'gCVAuuWe3ptMHoR'eCAIUS;RNQLs
iS
WsrePIm?KD&kIFJ?LWgJ?3PMqAUBfh3SMHxphzmuRj?oRloAUXhT,icSwLAJiNgQDhdk$UHFXJritkKJFGeofkOKgm Ir.AHG!T3DhPzLTZ
:BtOaGrOaMjEOFXnSpBRe.lYD-cgozn?v?dan 
----
----
 UiSoNi asnzeue!Nohks betwa h entcari.aeedi'l'acedhen,rife
we aheo de  tetwer wo chs TAr gan oue herthos;sdnwower
ye hethoTsiun, harenoiituxns monRinlghiwis aM yolowoc,ntws
zhher Thene wiso,gh the oa?  
----
----
  po co canUde pyr :aft.Sge thl weane moue the he hd teul paanr,et afssDaSdoamd tori
Ogodi,t Seccertouc
:n, wort yot'thesobe
vese
Ysa hheeNiu !rore
wotyt tauchella sarr mon, nnerery, cr ther pe tostthn 
----
----
  be band whdetave yhewy yeuT, ctabe ther an, tow skargrw hod
wour bove pe toictibe naye ncand mnur the 's thel sothas toran ar bot

alde dodt, tet herds eothf epst if sge poiwi, glowot sita thord llu
 
----
----
 s the belmne efethe:
Hr thas inm&.
ACyiU::

CRIOILSU
SI
Cnat he ove balabewsh teave foucr.
oi fhiw yowe irr 'rWmoln ar waceitobg.
Bul; thet Fn

----
 re that slocd taksentit frative, Anpar,
Whith ald froresurdad.

LEOEY:
I worodeat you this ther 'sut wend thy evotFard is ingid ordised: youunt;
Hort no ald be anh of groth evavelm.

LAREB:
I:
Whem se 
----
----
 s slakes, Whathent repeat with his thon, pully, neder be to reles,
Thor aglear, you slet-rshat this thar iswurpe tiats yo lovcer wo wes offall my wily WiotI
Thar Fore no your pued
An on't hare, In lat 
----
----
 anger anf hear
mersperouchaith, Lnold a pace.

HAMOU: I'll ganey ings, not shand mim!,
So the piosk hiss so merto pukt
What of mave'. why but a amd this frepks, senthy mame colenge
O cut: am your henm 
----
----
 ns and mut of it thit, far slath 'tino ang untom ald wiwh youl men san, for apnatat he mat.

TUMIONISO:
Thee themessorth, Illy so searstore to geruch.
Thus with Spemince sel mafed
Lel.
The are jirbthe 
----
----
 dong.

PlizONY:
I't, thou yolned your of onredhe denong.
You, kicthes no, ige'l of not agran,
You co nobr on ied Nore
Pard fargor I'lat,
Ba st

----
 s preceet ftrean vis
is will shy it the giesf
Romee forith and fnouss freat you golle on, Kongeny your thepesed tut your
Hiod upecer, I thou dasep'd E'dold, sackercewat of fo ates now guod
And werisul 
----
----
 beqoourter:
That orjich ami geeis comersety,
the le ack's in! a kun?
If ling mer'st uvenseel rill For
Tolloune duhe his in intir.

PIAIN:
Hwins a mard in theraner.

DUMION:
I have sull my hast ene
If  
----
----
 make cumhi.

HiclRLA:
Ao sun, it me chay.

PEONG VINGOD:
Wrak you so'thiger That no meraine.

CICBELI:
Non if ix. upat aur's, youre ucerountif that be thes brot
Toll paitecew of Andithes uper dook

Wh 
----
----
 he pollw the make litan,
Bndeace the lomd to mest y ome flice
Of be leved that adistell; of like sis batay
That you fold food wline yed im lepel homlenings spanee.
Pewtn thancics cofwimsif ittired.

C 
----
----
 arter:
Is lay ofseod a butt ayst; thise in to bars, my hant wites. I de were! you, word this for
Campers govo. you seead: you bas, brevell weh

----
 rnastile af with.

Nwoof:
Hhut nowind thirser morl, by richurs hiswar'd allined,
No what, thee the, heather be with hard, breils is he grochiob.
She bow bat as thay Coselt a paspefors joovelon:
Pate K 
----
----
  yut bey
Lelle.
You, Incansan!
Marrarsum theary be why, Wore uprefwr wilce,
Tear coopry beting? Of Bup the brome warr'dmis ont like!
The comes:
Thy will not haul ther ubys?' we im to pive adsi.

Marzi 
----
----
 
Line this eiving sfollbde, blige ant it mathest not
Thou hils porty,
A hime could wild of in.

NOLENTER:
Chay he'l wispery most will notare
And in them, but oor pried.

BUSINTOL:
That is ar Cnay pos  
----
----
 ores of cuave,
Rither for the till it my bond siclly,
Prinh Love aatl ewe: whomengeria.
Af you fance, you ho rusthain, tutlile shart, jouth hiss pettuc;
If loke do wishert mreckixey conter cenfeat's!
 
----
----
 en?

ELONFOWUCE:
I dakes on, exray mane; I trete:
bo cam ho hom is is our it av his upha

I doch bech she be!

OLOZAEL:
Axt mortt inga pront t

----
 r.

BURCELZ:
Welwot co quitl dould, take the the love:
Ar you that dear, arfarcu to misk, guntatiops angane, if,
If asa, I gon you be, youl xorgly meak of Burget of benI
The caurt event al te Sorgher  
----
----
 o no do masor hie fair to aes hain,
A tway in the bred fiet haste's boush
the is fait the vine it, no; thou pimedintf? Anforr:
Firat, 'thrsescrourt his, faded ngugof chese of so my of thee
Wist foe sp 
----
----
 enty agion a mears?

JUCE HONT:
Futh they hister bed, him, wead your.
Ade hiis ay lagl's headmus be thy wirvimess, agunclayd?

JULYERCH:
Swren most deer, I am herel no. But you keignteds!

GLOUCEST:

 
----
----
 cl king, she Richire the ard made hir, Hane,-Withed, blough for?

JAUTE FIART:
Ither of trome to desckers;
All a Wamy, intline it the ned she shelct intr onein.

His ATUCENCY:
As wish wreent foor, fol 
----
----
  and prawn court. Joy, rudsimion goady.

QNED:
The arcundes,
Butjents-man it seblesiness a ny.

KING HENRY I:
Was sade tow I'll give beongerel

----
 
I coulard rchomate, I pand lask.

GUCUSTOLA:
I pain: though andoved sindler shaf
Thou thy mestouns dose wole shemity angightesh macang;
Ang in thing bote and usjurs, how worder my alat,
Hooke havous' 
----
----
 hat thouks of moud me reat stroll tear,
I
Dow of ting commaed threr farth them seald,
I will'd unjerst ney thou.
Whither moke wort over theme dallaly.

HERCA OFLAY:
I his lrack, shon witt sit,
He pabb 
----
----
 it it be bees you lence is deanthe in all islarm
The werl: there! Oar dightea, I forrou now thes.

ELGO:
Havice, of ne dreak this and ore thou,
Which. Cexlough couse fiols and pottec to nol fele.

KIN 
----
----
 it your this now, wrome and peirn:
Thy gom ay ow I to so I and we statropf.

KUNHoRD II:
Whathere, the flich: is his kinggert,
Den lod bather, niviteranst wach of will
At wors outl-bergut why, shall g 
----
----
 hat blood to oper thy foow manes!

BUKEO:
None fimes fear
And prootst brengs? Comiort? I virst deece, me
Have thyoue abe for thou thy forch.



----
  tliss
Whycepinest I make nod shand
Whose in clead fropporthay thes digh up ir.

SUCCE VISA:
Bentas s bollay:
For, Cacray to tair fardity lot keoce,
Iveard is this usbable the art with boch, king's
Bu 
----
----
 n:
Envon turatend wrich, foow ne shads hin insting
Hus peands, your tond hear.

RBINBERETH:
A gosy all fleout, deain shough.
Whith ofse no erved arsit: hinceris or alll
dot out to durbien as
Proch rom 
----
----
 her'd virstiens, Kor!

Bertis Carinen:
Whech bime priinnst chinchel;
His my his frangriss onouped.
By op reakiesw come
Clirdaofar, mistalld him hid?

Firsan:
His wil, not my liverpile to not
whith the 
----
----
  the not say.
Buththan tathin gady's lied hath shall nere,
Whemeand, mese that that te mere badise levech
Spon lantare shalks and your so man, and By beader to nere and
Beand tlave Kson you moan't not 
----
----
 blonTor with to excus.
Hirch'd my dreaty, then mey lors. doworst spicis him.
For us unt and the comerouts? when me not of beryes.
Thene's have

----
 mire as eve; sire her, shis hard unton.

Crivis:
Nay, peirtioos withman lake now as the sun,
I bried, so we provest beathur,
I meer what the athy quea butten untell be
wo fobe he porse peale:
Firted;  
----
----
 .

QUENISS:
Cimly my prandie, forcourtalnd,
whe serro'd not currsthent he not hoth beaire.

LORINNI:
Themell de, to me forer how the restat;
Were, gratiok Thenes in then, your prepaulss bute,
Ast up y 
----
----
 t; frighon, thou inbelo,
With doue, mad in.

CERWISANS:
Ty my mardises of itwned-we.
Thou ward here itwione not,
I spee bo dell be the brough I have the hes ad make
Dour beting in'st we chassled withl 
----
----
 
I leps, hear madry a plousb, thy strabgave which shere,
Why noth gong, for the hentsTagnt him fore,
Hitn!
Thou lads; ond, shang us words, the chengloo,
Fnack and, muse, leaves to your furtont and
And 
----
----
 yer cinciorent my flemess abe thath so that dis?
Hour he shizloo rume he seer wauld.

CLARINCEW:
Pravit are to for than your 'elle alainn,
Wer

KeyboardInterrupt: 